In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

c:\users\eshaa\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
df = pd.read_csv("real_combine.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
df = df.dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1092 non-null   float64
 1   TM      1092 non-null   float64
 2   Tm      1092 non-null   float64
 3   SLP     1092 non-null   float64
 4   H       1092 non-null   float64
 5   VV      1092 non-null   float64
 6   V       1092 non-null   float64
 7   VM      1092 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 85.3 KB


In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [6]:
X.shape, y.shape

((1092, 8), (1092,))

In [19]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int("num_layers", 2, 20)):
        model.add(layers.Dense(units = hp.Int("units_" + str(i), 
                                             min_value=32, 
                                             max_value=512,
                                             step=32), 
                               kernel_initializer = "he_uniform", 
                               activation = "relu"))
        
    model.add(layers.Dense(1, activation="linear"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mean_absolute_error", 
        metrics=["mean_absolute_error"])
        
    return model

In [25]:
tuner = RandomSearch(
            build_model, 
            objective="val_mean_absolute_error", 
            max_trials=5, 
#             executions_per_trial=3, 
            directory="Trained Weights", 
            project_name="AQ Index")

In [26]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((764, 8), (328, 8), (764,), (328,))

In [28]:
tuner.search(X_train, y_train, 
            epochs=20, 
            validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 16s]
val_mean_absolute_error: 50.05214309692383

Best val_mean_absolute_error So Far: 45.113433837890625
Total elapsed time: 00h 01m 05s
INFO:tensorflow:Oracle triggered exit


In [29]:
tuner.results_summary()

Results summary
Results in Trained Weights\AQ Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 17
units_0: 288
units_1: 160
learning_rate: 0.0001
units_2: 288
units_3: 480
units_4: 64
units_5: 512
units_6: 32
units_7: 256
units_8: 96
units_9: 480
units_10: 480
units_11: 288
units_12: 448
units_13: 32
units_14: 320
units_15: 320
units_16: 64
units_17: 224
Score: 45.113433837890625
Trial summary
Hyperparameters:
num_layers: 12
units_0: 512
units_1: 256
learning_rate: 0.0001
units_2: 416
units_3: 512
units_4: 480
units_5: 256
units_6: 320
units_7: 96
units_8: 224
units_9: 96
units_10: 448
units_11: 128
units_12: 384
units_13: 416
units_14: 32
units_15: 192
units_16: 288
units_17: 352
Score: 50.05214309692383
Trial summary
Hyperparameters:
num_layers: 18
units_0: 288
units_1: 96
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: